# Binding Functions

You can create your own functions to be called when a model is called as well. Outside of the functions that OpenAI creates. Let's create our own function to get the current time.

In [ ]:
pip install -q openai pytz

In [1]:
import json
from datetime import datetime
import pytz

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_time(time_zone):
    """Get the current time in a given location"""
    time = datetime.now(pytz.timezone(time_zone))
    time_info = {
        "time_zone": time_zone,
        "time": str(time),
    }
    return json.dumps(time_info)

In [2]:
# define a function
functions = [
    {
        "name": "get_current_time",
        "description": "Get the current time given a location",
        "parameters": {
            "type": "object",
            "properties": {
                "time_zone": {
                    "type": "string",
                    "description": "The time zone of the location we want to find the time for",
                },
            },
            "required": ["time_zone"],
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the time in Minneapolis using a 24 hour clock?"
        # TRY CHANGING IT TO THIS AND SEE WHAT HAPPENS!
        #"content": "What's the size of the moon?"
    }
]

In [13]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    functions=functions
)

In [14]:

print(response.choices[0].message.function_call)
#args = response.choices[0].message.function_call.arguments
if response.choices[0].message.function_call is None:
    args = {}
else:
    args = json.loads(response.choices[0].message.function_call.arguments)

    # Let's create another message that has the role of "function"
    messages.append(
        {
            "role": "function", #note this role is "function" this is how the model knows this the response is from the function
            "name": "get_current_time",
            "content": get_current_time(args["time_zone"]),
        }
    )


None


In [15]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    functions=functions
)

print(json.dumps(messages, indent=2))
print (response.choices[0].message.content)

[
  {
    "role": "user",
    "content": "What's the size of the moon?"
  }
]
The Moon has a diameter of about 3,474 kilometers (2,159 miles), making it the fifth-largest natural satellite in the Solar System. Its surface area is approximately 37.9 million square kilometers (14.6 million square miles), which is about 7.4% of the Earth's total surface area. The Moon's volume is about 21.9 billion cubic kilometers (5.29 billion cubic miles), and its mass is about 7.35 x 10^22 kilograms, which is roughly 1/81st of Earth's mass.
